In [ ]:
import cv2
import numpy as np
from os import listdir
import operator
import csv
from sklearn.metrics import accuracy_score

data=[]
matchCount={}
light_purple =(120,70,70)
dark_purple =(170,255,255)
cntR,cntT,cntS,cntG=[],[],[],[]
count={'ring':0,'troph':0,'schizont':0,'gametocyte':0}
acc={'ring':0,'troph':0,'schizont':0,'gametocyte':0}
search_keys=['ring','troph','schizont','gametocyte']
classify={}
classify.clear()
detected=[]
iCount=0
undetected=0
actual=[]
predict=[]
# Read image
for filenames in listdir('/content/drive/My Drive/Final Year Project/Test'):
    im = cv2.imread("/content/drive/My Drive/Final Year Project/Test/"+filenames)
    iCount+=1
    cp=im
    #cv2.imshow("Original Image", im)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("Grayscale Image", gray)

    th,im = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
    #cv2.imshow("Negative Binary Image", im)


    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()
    # Change thresholds
    params.minThreshold = 10
    params.maxThreshold = 200
    # Filter by Area.
    params.filterByArea = True
    params.minArea = 1000
    # Filter by Color
    params.filterByColor = False
    params.blobColor=0
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.4
    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = .15
    # Filter by Inertia
    params.filterByInertia =  True
    params.minInertiaRatio = 0.01

    detector = cv2.SimpleBlobDetector_create(params)
    # Detect blobs.
    keypoints = detector.detect(im)
    im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # Show blobs
    edged = cv2.Canny(im, 30, 200)
    contours, hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(im, contours, -1, (255,0,0), 2)
    #cv2.imshow("Contour Image", im)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    if len(contours) < 105:
        #cv2.drawContours(cp, contours, -1,(0,0,255),2)
        ##cv2.imshow("Original",cp)
        #print(filenames,len(contours))
        print("\nMalaria detected in ",filenames)
        detected.append(1)
    ##cv2.imshow("Original",cp)
    # Show blobs
        ##cv2.imshow("Keypoints", im_with_keypoints)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    #for filenames in listdir('C:/Users/Dell/Desktop/Project/Data/Temp'):
        img1 = cv2.imread("/content/drive/My Drive/Final Year Project/Test/"+filenames)
        for sub in search_keys:
            if(filenames.find(sub))!=-1:
                actual.append(sub)
                if sub in count:
                    count[sub] += 1
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        hsv_img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(hsv_img1, light_purple, dark_purple)
        res1 = cv2.bitwise_and(img1, img1, mask=mask)

        #cv2.imshow("Mask1 Image", mask)
        #cv2.imshow("Result Image", res1)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #print(filenames)
        for filename in listdir('/content/drive/My Drive/Final Year Project/Sample'):
                img2 = cv2.imread("/content/drive/My Drive/Final Year Project/Sample/"+filename,cv2.IMREAD_GRAYSCALE+cv2.THRESH_OTSU)
                # ORB Detector
                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
                hsv_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)

                mask = cv2.inRange(hsv_img2, light_purple, dark_purple)
                res2 = cv2.bitwise_and(img2, img2, mask=mask)
                orb = cv2.ORB_create()
                kp1, des1 = orb.detectAndCompute(res1, None)
                kp2, des2 = orb.detectAndCompute(res2, None)
                # Brute Force Matching
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                matches = bf.match(des1, des2)
                matches = [m for m in matches if m.distance<63]
                #print(len(matches))
                matchCount[filename]=len(matches)
                matching_result = cv2.drawMatches(res1, kp1, res2, kp2, matches, None, flags=2)
                #cv2.imshow("Img1", res1)
                #cv2.imshow("Img2", res2)

                #cv2.imshow("Mask2 Image", mask)
                #cv2.imshow("Result Image", res2 )
                #cv2.imshow("Matching result", matching_result)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()

                #print(matchCount)
        sorted_d = dict(sorted(matchCount.items(), key=operator.itemgetter(1),reverse=True))

        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        first5pairs = {k: sorted_d[k] for k in list(sorted_d)[:5]}
        first5=dict(first5pairs)
        #print(list(sorted_d.keys())[:5])
        #print(first5pairs)
        for sKey in search_keys:
            if sKey=='ring':
                ring = dict(filter(lambda item: sKey in item[0], first5.items()))
                try:
                    classify['ring']=sum(ring.values())
                except:
                    print("ring0")
            elif sKey=='troph':
                troph = dict(filter(lambda item: sKey in item[0], first5.items()))
                try:
                    classify['troph']=sum(troph.values())
                except:
                    print("troph0")
            elif sKey=='schizont':
                schizont = dict(filter(lambda item: sKey in item[0], first5.items()))
                try:
                    classify['schizont']=sum(schizont.values())
                except:
                    print("schizont0")
            elif sKey=='gametocyte':
                gametocyte = dict(filter(lambda item: sKey in item[0], first5.items()))
                try:
                    classify['gametocyte']=sum(gametocyte.values())
                except:
                    print("gametocyte0")
        #print(classify)
        classified = max(classify, key=classify.get)
        #print(classified+" == "+filenames)
        if (filenames.find(classified) != -1):
            print (filenames+" classified accurately as",classified)

            if classified=='ring': cntR.append(1)
            elif classified=='troph': cntT.append(1)
            elif classified=='schizont': cntS.append(1)
            elif classified=='gametocyte': cntG.append(1)

            data.append(tuple([filenames,'1',classified]))
        else:
            print (filenames+" classified wrongly as",classified)
            data.append(tuple([filenames,'0',classified]))
        predict.append(classified)

    else:
        print("\nMalaria undetected in ",filenames)
        detected.append(0)
    with open('outputCopy.csv', 'w', newline='') as file:
        writer = csv.writer(file,delimiter=',')
        writer.writerows(data)

#totalMatch=sum(cntR)+sum(cntT)+sum(cntS)+sum(cntG)
totalTemp=sum(count.values())
#print(count,detected)
print("\n\nDetection Accuracy:",(sum(detected)/iCount)*100)
#print(sum(cntR),sum(cntT),sum(cntS),sum(cntG))
#print('totalMatch =',totalMatch)
#print('totalTemp =',totalTemp)
#print("Detection Accuracy =",(detected/iCount)*100)
#print("Undetected Accrcay =",(undetected/iCount)*100)
#print('Classification Accuracy=',(totalMatch/totalTemp)*100)
#Acc=(totalMatch/totalTemp)*100
#print(actual)
#print(predict)
#results = confusion_matrix(actual, predict)
#print('Confusion Matrix :')
#print(results)
print("\nAccuracy Score for classification :",accuracy_score(actual, predict)*100)


Malaria detected in  Pf_rings_thinB.jpg
Pf_rings_thinB.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinD.jpg
Pf_rings_thinD.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinE.jpg
Pf_rings_thinE.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinF.jpg
Pf_rings_thinF.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinG.jpg
Pf_rings_thinG.jpg classified wrongly as gametocyte

Malaria detected in  Pf_rings_thinH.jpg
Pf_rings_thinH.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinI.jpg
Pf_rings_thinI.jpg classified accurately as ring

Malaria detected in  Pf_rings_thinJ.jpg
Pf_rings_thinJ.jpg classified accurately as ring

Malaria detected in  Pf_troph_thinC.jpg
Pf_troph_thinC.jpg classified wrongly as gametocyte

Malaria detected in  Pf_gametocyte_thinD.jpg
Pf_gametocyte_thinD.jpg classified accurately as gametocyte

Malaria detected in  Pf_gametocyte_thinE.jpg
Pf_gametocyte_thinE.jpg classifi